# Expression trees in PyBaMM

The basic data structure that PyBaMM uses to express models is an expression tree. This data structure encodes a tree representation of a given equation. The expression tree is used to encode the equations of both the original symbolic model, and the discretised equations of that model. Once discretised, the model equations are then passed to the solver, which must then evaluate the discretised expression trees in order to perform the time-stepping.

The expression tree must therefore satisfy three requirements:
1. To encode the model equations, it must be able to encode an arbitrary equation, including unary and binary operators such as `*`, `-`, spatial gradients or divergence, symbolic parameters, scalar, matrices and vectors.
2. To perform the time-stepping, it must be able to be evaluated, given the current state vector $\mathbf{y}$ and the current time $t$
3. For solvers that require it, its gradient with respect to a given variable must be able to be evaluated (once again given $\mathbf{y}$ and $t$)

As an initial example, the code below shows how to construct an expression tree of the equation $2y(1 - y) + t$. We use the `pybamm.StateVector` to represent $\mathbf{y}$, which in this case will be a vector of size 1. The time variable $t$ is already provided by PyBaMM and is of class `pybamm.Time`.

In [1]:
%pip install pybamm -q    # install PyBaMM if it is not installed
import pybamm
import numpy as np

y = pybamm.StateVector(slice(0,1))
t = pybamm.t
equation = 2*y * (1 - y) + t
equation.visualise('expression_tree1.png')

Note: you may need to restart the kernel to use updated packages.


![](expression_tree1.png)

Once the equation is constructed, we can evaluate it at a given $t=1$ and $\mathbf{y}=\begin{pmatrix} 2 \end{pmatrix}$.

In [2]:
equation.evaluate(1, np.array([2]))

array([[-3.]])

We can also calculate the expression tree representing the gradient of the equation with respect to $t$,

In [3]:
diff_wrt_equation = equation.diff(t)
diff_wrt_equation.visualise('expression_tree2.png')

![](expression_tree2.png)


...and evaluate this expression,

In [4]:
diff_wrt_equation.evaluate(t=1, y=np.array([2]), y_dot=np.array([2]))

array([[-11.]])

## The PyBaMM Pipeline

Proposing, parameter setting and discretising a model in PyBaMM is a pipeline process, consisting of the following steps:

1. The model is proposed, consisting of equations representing the right-hand-side of an ordinary differential equation (ODE), and/or algebraic equations for a differential algebraic equation (DAE), and also associated boundary condition equations
2. The parameters present in the model are replaced by actual scalar values from a parameter file, using the [`pybamm.ParamterValues`](https://pybamm.readthedocs.io/en/latest/source/parameters/parameter_values.html) class
3. The equations in the model are discretised onto a mesh, any spatial gradients are replaced with linear algebra expressions and the variables of the model are replaced with state vector slices. This is done using the [`pybamm.Discretisation`](https://pybamm.readthedocs.io/en/latest/source/spatial_methods/discretisation.html) class.

## Stage 1 - Symbolic Expression Trees

At each stage, the expression tree consists of certain types of nodes. In the first stage, the model is first proposed using [`pybamm.Parameter`](https://pybamm.readthedocs.io/en/latest/source/expression_tree/parameter.html), [`pybamm.Variable`](https://pybamm.readthedocs.io/en/latest/source/expression_tree/variable.html), and other [unary](https://pybamm.readthedocs.io/en/latest/source/expression_tree/unary_operator.html) and [binary](https://pybamm.readthedocs.io/en/latest/source/expression_tree/binary_operator.html) operators (which also includes spatial operators such as [`pybamm.Gradient`](https://pybamm.readthedocs.io/en/latest/source/expression_tree/unary_operator.html#pybamm.Gradient) and [`pybamm.Divergence`](https://pybamm.readthedocs.io/en/latest/source/expression_tree/unary_operator.html#pybamm.Divergence)). For example, the right hand side of the equation

$$\frac{d c}{dt} = D \nabla \cdot \nabla c$$

can be constructed as an expression tree like so:

In [5]:
D = pybamm.Parameter('D')
c = pybamm.Variable('c', domain=['negative electrode'])

dcdt = D * pybamm.div(pybamm.grad(c))
dcdt.visualise('expression_tree3.png')

![](expression_tree3.png)


## Stage 2 - Setting parameters

In the second stage, the `pybamm.ParameterValues` class is used to replace all the parameter nodes with scalar values, according to an input parameter file. For example, we'll use a this class to set $D = 2$

In [6]:
parameter_values = pybamm.ParameterValues({'D': 2})
dcdt = parameter_values.process_symbol(dcdt)
dcdt.visualise('expression_tree4.png')

![](expression_tree4.png)

## Stage 3 - Linear Algebra Expression Trees

The third and final stage uses the `pybamm.Discretisation` class to discretise the spatial gradients and variables over a given mesh. After this stage the expression tree will encode a linear algebra expression that can be evaluated given the state vector $\mathbf{y}$ and $t$.

**Note:** for demonstration purposes, we use a dummy discretisation below. For a more complete description of the `pybamm.Discretisation` class, see the example notebook [here](https://github.com/pybamm-team/PyBaMM/blob/develop/examples/notebooks/spatial_methods/finite-volumes.ipynb).

In [7]:
from tests import get_discretisation_for_testing
disc = get_discretisation_for_testing()
disc.y_slices = {c.id: [slice(0, 40)]}
dcdt = disc.process_symbol(dcdt)
dcdt.visualise('expression_tree5.png')

ImportError: cannot import name 'get_discretisation_for_testing' from 'tests' (/home/ferranbrosa/PyBaMM/env/lib/python3.7/site-packages/tests/__init__.py)

![](expression_tree5.png)

After the third stage, our expression tree is now able to be evaluated by one of the solver classes. Note that we have used a single equation above to illustrate the different types of expression trees in PyBaMM, but any given models will consist of many RHS or algebraic equations, along with boundary conditions. See [here](https://github.com/pybamm-team/PyBaMM/tree/develop/examples/notebooks/Creating%20Models) for more details of PyBaMM models.

## References

The relevant papers for this notebook are:

In [8]:
pybamm.print_citations()

[1] Charles R. Harris, K. Jarrod Millman, Stéfan J. van der Walt, Ralf Gommers, Pauli Virtanen, David Cournapeau, Eric Wieser, Julian Taylor, Sebastian Berg, Nathaniel J. Smith, and others. Array programming with NumPy. Nature, 585(7825):357–362, 2020. doi:10.1038/s41586-020-2649-2.
[2] Valentin Sulzer, Scott G. Marquis, Robert Timms, Martin Robinson, and S. Jon Chapman. Python Battery Mathematical Modelling (PyBaMM). ECSarXiv. February, 2020. doi:10.1149/osf.io/67ckj.

